In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='128f549d-5ca1-4662-a5fd-39b2838aad90', project_access_token='p-2+oaXfM7ykge3PbsrLWC66uA==;SyTxfG5nx+NjLk8uReDY+A==:jZslqrGTm0CCZYM9qedq4RrjP7BuU+GqO1+LBJTqRUeh5Gb+VvCBrnoTdCI2Ifeepa/KzCk4iJymoQrtpaqvfOd8i+Bc3onpyA==')
pc = project.project_context

from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space({'token':'p-2+oaXfM7ykge3PbsrLWC66uA==;SyTxfG5nx+NjLk8uReDY+A==:jZslqrGTm0CCZYM9qedq4RrjP7BuU+GqO1+LBJTqRUeh5Gb+VvCBrnoTdCI2Ifeepa/KzCk4iJymoQrtpaqvfOd8i+Bc3onpyA=='})

In [2]:
!pip install -U ibm-watson-machine-learning
!pip install langchain
!pip install ragas
!pip install faiss-cpu
!pip install datasets
!pip install sentence-transformers
!pip install python-dotenv

In [3]:
IBM_CLOUD_URL  = 'https://us-south.ml.cloud.ibm.com/' # Change this according to your region. For Dallas keep the same.
PROJECT_ID     = '128f549d-5ca1-4662-a5fd-39b2838aad90'
WATSONX_APIKEY = '14qK2UldM1i25REfFm3HcH9y23_NVL-6taz3pgKjrnC9'

In [4]:
# Generate_Response.py — will load a LLM from watsonx.ai platform and generate a response by using the defined prompt. 
# You can modify the LLM being used and the prompt according to your model. This script already has prompts for Llama 2 70b chat and IBM Granite 13b instruct models.

from ibm_watson_machine_learning.foundation_models import Model
from dotenv import load_dotenv
import os

class GenerateLLMResponse:
    def __init__(self, model_id    = "ibm/granite-13b-instruct-v2",
                 generation_params = {
                "decoding_method"    : "sample",
                "max_new_tokens"     : 512,
                "stop_sequences"     : ["\n\n\n"],
                "temperature"        : 0.1,
                "top_k"              : 50,
                "top_p"              : 1,
                "repetition_penalty" : 1
                }):
        # Load credentials
        load_dotenv()
        api_key       = WATSONX_APIKEY #os.getenv("WATSONX_APIKEY", None)
        ibm_cloud_url = IBM_CLOUD_URL  #os.getenv("IBM_CLOUD_URL", None)
        project_id    = PROJECT_ID     #os.getenv("PROJECT_ID", None)
        creds = {
                "url"    : ibm_cloud_url,
                "apikey" : api_key 
        }
        # Initialize model 
        self.model = Model(model_id, params=generation_params, credentials=creds, project_id=project_id)

    # Create the Prompt
    def build_prompt(self,question,context):
        # Prompt Template for Llama 2 70b chat
        #         prompt = f'''
        # [INST]
        # Context: {context}
        # - Take the context above and use that to answer questions in a detailed and professional way..
        # - if you dont know the answer just say ” i dont know “.
        # - refrain from using any other knowledge other than the text provided.
        # - don't mention that you are answering from the text, impersonate as if this is coming from your knowledge
        # - For the questions whose answer is not available in the provided context, just say please ask a relevant question or frame the question more clearly
        # Question: {question}?
        # [/INST]
        # '''
    
        # Prompt Template for IBM Granite 13b instruct
        prompt=f'''Given the document and the current conversation between a user and an agent, your task is as follows: Answer any user query by using information from the document. The response should be detailed and should be written only in English.

                # DOCUMENT: {context}
                # DIALOG: USER: {question}
                '''
        return prompt

    def answer_based_on_context(self,query,context):
        prompt = self.build_prompt(query, context)
        # print(prompt)
        result = self.model.generate(prompt)
        result = result["results"][0]["generated_text"]
        return result.strip()

In [5]:
# ragas_langchain_customization.py — Contains a little customization for the Ragas evaluation using langchain tailored to Watsonx LLM 
# because the original code uses OpenAI credentials. Create this in your project folder.
# ragas: rag assessment

#!pip install ibm_watson_machine_learning
import typing as t

from ragas.llms import LangchainLLM
from ragas.llms.langchain import MULTIPLE_COMPLETION_SUPPORTED

from ragas.async_utils import run_async_tasks
from ragas.llms.langchain import _compute_token_usage_langchain, isBedrock

from langchain.callbacks.base import Callbacks
from langchain.prompts import ChatPromptTemplate

try:
    from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
except ImportError:
    raise ImportError(
        "ibm_watson_machine_learning must be installed to use this function. "
        "Please, install it with `pip install ibm_watson_machine_learning`."
    )

from langchain.schema import LLMResult
from langchain.llms.base import BaseLLM



def isWatsonx(llm: WatsonxLLM) -> bool:
    return isinstance(llm, WatsonxLLM)


# have to specify it twice for runtime and static checks
MULTIPLE_COMPLETION_SUPPORTED.append(WatsonxLLM)

class CustomizedLangchainLLM(LangchainLLM):

    def generate(
        self,
        prompts: list[ChatPromptTemplate],
        n: int = 1,
        temperature: float = 1e-8,
        callbacks: t.Optional[Callbacks] = None,
    ) -> LLMResult:
        
        ######## Change for watsonX #########
        if isWatsonx(self.llm):
            return self._generate_multiple_completions_watsonx(prompts, callbacks)
        ########################################################################

        # set temperature to 0.2 for multiple completions
        temperature = 0.2 if n > 1 else 1e-8
        if isBedrock(self.llm) and ("model_kwargs" in self.llm.__dict__):
            self.llm.model_kwargs = {"temperature": temperature}
        else:
            self.llm.temperature = temperature

        if self.llm_supports_completions(self.llm):
            return self._generate_multiple_completions(prompts, n, callbacks)
        else:  # call generate_completions n times to mimic multiple completions
            list_llmresults = run_async_tasks(
                [self.generate_completions(prompts, callbacks) for _ in range(n)]
            )

            # fill results as if the LLM supported multiple completions
            generations = []
            for i in range(len(prompts)):
                completions = []
                for result in list_llmresults:
                    completions.append(result.generations[i][0])
                generations.append(completions)

            llm_output = _compute_token_usage_langchain(list_llmresults)
            return LLMResult(generations=generations, llm_output=llm_output)


    # Add function for watsonxX
    def _generate_multiple_completions_watsonx(
            self,
            prompts: list[ChatPromptTemplate],
            callbacks: t.Optional[Callbacks] = None,
        ) -> LLMResult:
            self.langchain_llm = t.cast(WatsonxLLM, self.langchain_llm)

            if isinstance(self.llm, BaseLLM):
                ps = [p.format() for p in prompts]
                result = self.llm.generate(ps, callbacks=callbacks)
            else:  # if BaseChatModel
                ps = [p.format_messages() for p in prompts]
                result = self.llm.generate(ps, callbacks=callbacks)
            return result

In [6]:
# Import libraries

import os
from dotenv import load_dotenv
import pandas as pd
#from langchain.llms import WatsonxLLM
from langchain.chains import HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
import langchain
import ast

from ragas.metrics import faithfulness,context_precision,context_recall,context_relevancy,AnswerCorrectness
from datasets import Dataset
from ragas import evaluate

from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

from typing import Dict, Union

In [48]:
# Load dataset

# assets = wslib.list_connections()
# wslib.show(assets)
# connprops = wslib.get_connection(assets[0])
# wslib.show(connprops)

# List all assets in the project
# notebooks = wslib.assets.list_assets('notebook')
# wslib.show(notebooks)

# asstes = wslib.assets.list_assets("asset")
# wslib.show(asstes)

dataset_info = wslib.assets.list_assets("data_asset", name="dataset.csv")

ds_1 = 1
ds_2 = 0
if len(dataset_info) > 0:
    dataset_name = dataset_info[0]['name']
    print(dataset_name + ' exists!')
    
    my_file = wslib.load_data('dataset.csv')
    my_file.seek(0)
    df = pd.read_csv(my_file)
else:
    !pip install pyarrow
    if ds_1: # If you are using the MS-FIQA dataset
        my_file = wslib.load_data('dataset_1.parquet')
        my_file.seek(0)
        df_all = pd.read_parquet(my_file, engine='pyarrow')

        # Keep only the required columns
        df = df_all[['question','contexts', 'ground_truths']]

        # Make sure that these two columns are a list
        #df["ground_truths"] = [ast.literal_eval(gt) for gt in df["ground_truths"]]
        #df["contexts"]      = [ast.literal_eval(ctx) for ctx in df["contexts"]]
        df["ground_truths"] = [gt for gt in df["ground_truths"]]
        df["contexts"]      = [ctx for ctx in df["contexts"]]
    else:  # If you are using the MS-MARCO dataset 
        my_file = wslib.load_data('dataset_2.parquet')
        my_file.seek(0)
        df = pd.read_parquet(my_file, engine='pyarrow')

        #Keep only those rows where thre is atleast an answer for a query
        df = df[df['answers'].apply(len) > 0]

        # Some pre-processing to bring it into right format.
        df.loc[:, 'passages'] = df.passages.apply(lambda x : x['passage_text'])

        # Keep only required columns
        df = df[['query', 'passages', 'answers']].reset_index(drop=True)

        # Renaming columns because RAGAS library expect only these names
        df.rename(columns = {'answers':'ground_truths', 'query':'question', 'passages': 'contexts'}, inplace =True)

    # Save to csv for further use
    project.save_data(file_name = "dataset.csv",data = df.to_csv(index=False))
    #df.to_csv('ms-marco-200-rows.csv', index = False)

# Using only 30 rows. You can modify accordingly
df = df[:10]

print('df.shape: ', df.shape)
print(df.head())    

dataset.csv exists!
df.shape:  (10, 3)
                                            question  \
0  How to deposit a cheque issued to an associate...   
1  Can I send a money order from USPS as a business?   
2  1 EIN doing business under multiple business n...   
3         Applying for and receiving business credit   
4               401k Transfer After Business Closure   

                                            contexts  \
0  ['Just have the associate sign the back and th...   
1  ['Sure you can.  You can fill in whatever you ...   
2  ['You\'re confusing a lot of things here. Comp...   
3  ['Set up a meeting with the bank that handles ...   
4  ['The time horizon for your 401K/IRA is essent...   

                                       ground_truths  
0  ["Have the check reissued to the proper payee....  
1  ["Sure you can.  You can fill in whatever you ...  
2  ["You're confusing a lot of things here. Compa...  
3  ['"I\'m afraid the great myth of limited liabi...  
4  ["You sho

In [49]:
# Create chunks from the text: You can modify the chunk size and chunk overlap numbers accordingly

def text_to_chunks(texts: str,
                   chunk_length: int = 100,
                   chunk_overlap: int = 25) -> list:
    """
    Splits the text into equally distributed chunks with 25-word overlap.
    Args:
        texts (str): Text to be converted into chunks.
        chunk_length (int): Maximum number of words in each chunk.
        chunk_overlap (int): Number of words to overlap between chunks.
    """
    words = texts.split(' ')
    n = len(words)
    chunks = []
    chunk_number = 1
    i = 0
    while i < n:  # Corrected the length check
        chunk = words[i: min(i + chunk_length, n)]
        i = i + chunk_length - chunk_overlap
        #print(len(chunk))
        chunk = ' '.join(chunk).strip()
        chunks.append({"text": chunk})
        chunk_number += 1
    return chunks


# Call the above module on the contexts column
if ds_1:
    df['chunks'] = df['contexts'].apply(lambda x: [i['text'] for i in text_to_chunks(x[0])])
else:
    # Use the below line if using MS-MARCO dataset
    #df['chunks'] = df['contexts'].apply(lambda x: [i['text'] for i in text_to_chunks('\n'.join(x))])
    df['chunks'] = df['contexts'].apply(lambda x: [i['text'] for i in text_to_chunks(x[0])])
df.head()

,question,contexts,ground_truths,chunks
0,How to deposit a cheque issued to an associate...,['Just have the associate sign the back and th...,"[""Have the check reissued to the proper payee....",[[]
1,Can I send a money order from USPS as a business?,['Sure you can. You can fill in whatever you ...,"[""Sure you can. You can fill in whatever you ...",[[]
2,1 EIN doing business under multiple business n...,['You\'re confusing a lot of things here. Comp...,"[""You're confusing a lot of things here. Compa...",[[]
3,Applying for and receiving business credit,['Set up a meeting with the bank that handles ...,"['""I\'m afraid the great myth of limited liabi...",[[]
4,401k Transfer After Business Closure,['The time horizon for your 401K/IRA is essent...,"[""You should probably consult an attorney. How...",[[]


In [50]:
# Create Document objects for the chunks to store into a vector database. (Here Faiss)

data = []

for i in range(len(df)):
    for j in df['chunks'][i]:
        doc = Document(
            metadata={
                "question": df['question'][i],
            },
            page_content=j)
        data.append(doc)
        
print(data[0])  
print('len(data): ', len(data))

page_content='[' metadata={'question': 'How to deposit a cheque issued to an associate in my business into my business account?'}
len(data):  10


In [51]:
# Initialize the embedding and the watsonx model

# Initialie the embedding model
embedder     = "intfloat/e5-large-v2"
model_kwargs = {'device': 'cpu'}

embeddings_model = HuggingFaceEmbeddings(
    model_name   = embedder,
    model_kwargs = model_kwargs
)

# Initialize the watsonx model
parameters = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.MAX_NEW_TOKENS: 200,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.2,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1,
}

load_dotenv()
api_key       = WATSONX_APIKEY #os.getenv("WATSONX_APIKEY", None)
ibm_cloud_url = IBM_CLOUD_URL  #os.getenv("IBM_CLOUD_URL", None)
project_id    = PROJECT_ID     #os.getenv("PROJECT_ID", None)

creds = {
        "url"    : ibm_cloud_url,
        "apikey" : api_key 
}
print('api_key: ', api_key)
print('ibm_cloud_url: ', ibm_cloud_url)
print('project_id: ', project_id)

# Initialize model 
model = Model(model_id="ibm/granite-13b-instruct-v2", params=parameters, credentials=creds, project_id=project_id)
llm = WatsonxLLM(model=model)

api_key:  14qK2UldM1i25REfFm3HcH9y23_NVL-6taz3pgKjrnC9
ibm_cloud_url:  https://us-south.ml.cloud.ibm.com/
project_id:  128f549d-5ca1-4662-a5fd-39b2838aad90


In [52]:
# Setup the Hypothetical answer retriever

print('Creating Faiss vectorstore ...')
prompt = 'Please write a paragraph to answer the below question.\nQuestion: {QUESTION}'
prompt_template = PromptTemplate.from_template(prompt)


# Generate hypothetical document for query using zero shot LLM call, and then embed that using the embeddings model defined above.
embeddings = HypotheticalDocumentEmbedder.from_llm(llm,
                                                   embeddings_model,
                                                   custom_prompt=prompt_template,
                                                   verbose = False
                                                   )
print('embeddings.llm_chain.prompt: ', embeddings.llm_chain.prompt)
langchain.debug = True  # Keep it False if you do not want tons of prompt messages

# Putting the data into vector store
vectorstore = FAISS.from_documents(data, embeddings)

# Retrieve similar documents to hypothetical answer from the vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}, verbose = False)
print('Faiss vectorstore created successfully ...')

Creating Faiss vectorstore ...
embeddings.llm_chain.prompt:  input_variables=['QUESTION'] template='Please write a paragraph to answer the below question.\nQuestion: {QUESTION}'
Faiss vectorstore created successfully ...


In [53]:
# Initialize an LLM from watsonx.ai platform for response generation

generation_params = {
    "decoding_method"   : "greedy",
    "max_new_tokens"    : 512,
    "repetition_penalty": 1
}

# You can change the model being used by modifying the model_id
model_id     = "ibm/granite-13b-instruct-v2"
response_llm = GenerateLLMResponse(model_id=model_id, generation_params=generation_params)

In [54]:
# Test a sample query to fetch context according to HyDE and then a LLM response using that context

# Testing a sample query from the dataset
query = "Can I send a money order from USPS as a business?"
docs  = retriever.get_relevant_documents(query, verbose=False)

# Print the context
for i in docs:
    print(i.page_content)

# Generate the LLM Response

context  = [i.page_content for i in docs] # Join the list into a single text context.

response = response_llm.answer_based_on_context(query=query, context=context)
print(response)

[llm/start] [1:llm:WatsonxLLM] Entering LLM run with input:
{
  "prompts": [
    "Please write a paragraph to answer the below question.\nQuestion: Can I send a money order from USPS as a business?"
  ]
}
[llm/end] [1:llm:WatsonxLLM] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, you can send a money order from USPS as a business.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[
[
[
AGENT: Yes, you can. You can find the information on the USPS website.


In [55]:
# Fetch the context and LLM response for the whole dataset

#!pip install openpyxl

print('Answering the whole dataset started ...')
# Only keeping the question and ground truths
df = df[['question', 'ground_truths']] 

# Retrieving contexts for the whole dataset using the Hypothetical retriever
df['contexts'] = df['question'].apply(lambda x : [i.page_content for i in retriever.get_relevant_documents(x)])

# Use the below line if using MS-MARCO dataset
# df['contexts'] = df['question'].apply(lambda x : [i.page_content for i in retriever.get_relevant_documents(x)])

# Generating LLM response to the queries using the retrieved contexts
df['answer'] = df.apply(lambda row: response_llm.answer_based_on_context(query=row['question'], context=row['contexts']), axis=1)

# Save the output for further use
#df.to_excel('hyde-output-baseline.xlsx', index = False) # You might need openpyxl. Install using !pip install openpyxl
df.head()
print('Answering process completed!')

Answering the whole dataset started ...
[llm/start] [1:llm:WatsonxLLM] Entering LLM run with input:
{
  "prompts": [
    "Please write a paragraph to answer the below question.\nQuestion: How to deposit a cheque issued to an associate in my business into my business account?"
  ]
}
[llm/end] [1:llm:WatsonxLLM] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "You can deposit a cheque issued to an associate in your business into your business account by endorsing the cheque and signing it.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[llm/start] [1:llm:WatsonxLLM] Entering LLM run with input:
{
  "prompts": [
    "Please write a paragraph to answer the below question.\nQuestion: Can I send a money order from USPS as a business?"
  ]
}
[llm/end] [1:llm:WatsonxLLM] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, you can send a money order f

In [56]:
# Evaluate using Ragas
# A method to initialize LLM from watsonx.ai for Ragas evaluation.

def get_watsonxllm_wrapper(
        model_id:str = "meta-llama/llama-2-70b-chat",
        parameters:Dict = {
            GenParams.DECODING_METHOD: DecodingMethods.GREEDY
        }
    )->WatsonxLLM:
    """
    This Function will return the watsonx Lanchain wrapper.
    """
    llama_model = Model(
        model_id=model_id,
        params=parameters,
        credentials=creds,
        project_id=project_id)
    return WatsonxLLM(model=llama_model)

Evaluation of the model

Ragas: Ragas is a framework that helps you evaluate your Retrieval Augmented Generation (RAG) pipelines. RAG denotes a class of LLM applications that use external data to augment the LLM’s context.

The metrics that will be used to evaluate our HyDE framework would be:
1_Faithfulness
This measures the factual consistency of the generated answer against the given context. It is calculated from answer and retrieved context. The answer is scaled to (0,1) range. Higher the better.
The generated answer is regarded as faithful if all the claims that are made in the answer can be inferred from the given context. To calculate this a set of claims from the generated answer is first identified. Then each one of these claims are cross checked with given context to determine if it can be inferred from given context or not.

2_Context Precision
Context Precision is a metric that evaluates whether all of the ground-truth relevant items present in the contexts are ranked higher or not. Ideally all the relevant chunks must appear at the top ranks. This metric is computed using the question and the contexts, with values ranging between 0 and 1, where higher scores indicate better precision.

3_Context Relevancy
This metric gauges the relevancy of the retrieved context, calculated based on both the question and contexts. The values fall within the range of (0, 1), with higher values indicating better relevancy.
Ideally, the retrieved context should exclusively contain essential information to address the provided query. To compute this, we initially estimate the value of |S| by identifying sentences within the retrieved context that are relevant for answering the given question.

In [57]:
# A method to specify LLM for each metrics and then evaluate on a dataset

def _get_ragas_score(df, dataset:Union[Dataset,None]=None, sample:Union[int,None]=None):
    
    # Initialize the model using the wrapper method defined above. You can initialize diffrent models for each metrics. I have used same for all.
    llama_llm = get_watsonxllm_wrapper()
    
    # Initialize the Ragas class 
    ragas_model = CustomizedLangchainLLM(llm=llama_llm)
    #df = df[['question','contexts','answer', 'ground_truths']]    # Specify LLM for each metrics (Here same for all)
    df = df[['question','contexts','answer', 'ground_truths']]    # Specify LLM for each metrics (Here same for all)
    
    faithfulness.llm      = ragas_model
    context_precision.llm = ragas_model
    context_relevancy.llm = ragas_model
    
    if sample:
        print(f"Dataset is not passed. Creating sample results {sample=} from explodinggradients/fiqa")
        result = evaluate(
        dataset.select(range(sample)),  # showing only 5 for demonstration
            metrics=[faithfulness, context_precision, context_relevancy],
        )
    else:
        result = evaluate(dataset, metrics=[faithfulness, context_precision, context_relevancy ])
    return result

In [62]:
# Prepare the dataset for evaluation

df = df[['question','contexts','answer', 'ground_truths']]

# If loading from the saved excel then:
# Making sure ground_truths and contexts columns have list values and not strings

#df["ground_truths"] = [ast.literal_eval(gt) for gt in df["ground_truths"] ]
#df["contexts"] = [ast.literal_eval(ctx) for ctx in df["contexts"]]
df["ground_truths"] = [gt for gt in df["ground_truths"] ]
df["contexts"] = [ctx for ctx in df["contexts"]]

df.head()

,question,contexts,answer,ground_truths
0,How to deposit a cheque issued to an associate...,"[[, [, []",AGENT: You can deposit the cheque by using the...,[Have the check reissued to the proper payee.J...
1,Can I send a money order from USPS as a business?,"[[, [, []","AGENT: Yes, you can. You can find the informat...",[Sure you can. You can fill in whatever you w...
2,1 EIN doing business under multiple business n...,"[[, [, []","AGENT: Sorry, I don't know the answer to that ...",[You're confusing a lot of things here. Compan...
3,Applying for and receiving business credit,"[[, [, []",AGENT: Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili..."
4,401k Transfer After Business Closure,"[[, [, []",AGENT: 401k Transfers are not available for Cl...,[You should probably consult an attorney. Howe...


In [63]:
# Create a Dataset object from pandas (This is the input format required for the RAGAS library)

hyde_dataset = Dataset.from_pandas(df, preserve_index=False)
print('hyde_dataset.shape: ', hyde_dataset.shape)
print(hyde_dataset.data)

hyde_dataset.shape:  (10, 4)
InMemoryTable
question: string
contexts: list<item: string>
  child 0, item: string
answer: string
ground_truths: list<item: string>
  child 0, item: string
----
question: [["How to deposit a cheque issued to an associate in my business into my business account?","Can I send a money order from USPS as a business?","1 EIN doing business under multiple business names","Applying for and receiving business credit","401k Transfer After Business Closure","What are the ins/outs of writing equipment purchases off as business expenses in a home based business?","Can a entrepreneur hire a self-employed business owner?","Intentions of Deductible Amount for Small Business","How can I deposit a check made out to my business into my personal account?","Filing personal with 1099s versus business s-corp?"]]
contexts: [[["[","[","["],["[","[","["],...,["[","[","["],["[","[","["]]]
answer: [["AGENT: You can deposit the cheque by using the deposit slip at your bank.","AGENT: 

In [64]:
# Call the evaluation method defined above on the dataset

print('Scoring the Answers ...')
ragas_scores = _get_ragas_score(df, dataset=hyde_dataset)
print(ragas_scores)
print('Scoring completed!')

Scoring the Answers ...
evaluating with [faithfulness]
[chain/start] [1:chain:ragas_faithfulness] Entering Chain run with input:
{}


  0%|          | 0/1 [00:00<?, ?it/s]

[chain/start] [1:chain:batch] Entering Chain run with input:
{}
[llm/start] [1:chain:batch > 2:llm:WatsonxLLM] Entering LLM run with input:
{
  "prompts": [
    "Human: Create one or more statements from each sentence in the given answer.\n\nquestion: Who was  Albert Einstein and what is he best known for?\nanswer: He was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time. He was best known for developing the theory of relativity, he also made important contributions to the development of the theory of quantum mechanics.\nstatements in json:\n{\n    \"statements\": [\n        \"Albert Einstein was born in Germany.\",\n        \"Albert Einstein was best known for his theory of relativity.\"\n    ]\n}\n\nquestion: Cadmium Chloride is slightly soluble in this chemical, it is also called what?\nanswer: alcohol\nstatements in json:\n{\n    \"statements\": [\n        \"Cadmium Chloride is slightly soluble in alcohol.

[llm/end] [1:chain:batch > 2:llm:WatsonxLLM] [12.24s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n{\n    \"statements\": [\n        \"You can deposit the cheque by",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[llm/end] [1:chain:batch > 3:llm:WatsonxLLM] [12.24s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n{\n    \"statements\": [\n        \"USPS allows businesses to send money",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[llm/end] [1:chain:batch > 4:llm:WatsonxLLM] [12.24s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n{\n    \"statements\": []\n}\n\nquestion: What is the chemical symbol",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[llm/end] [1:chai

[llm/end] [1:chain:batch > 2:llm:WatsonxLLM] [1.08s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\n    \"statements\": [\n        \"USPS allows businesses to send money\"",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:batch] [1.08s] Exiting Chain run with output:
{}
[chain/start] [1:chain:batch] Entering Chain run with input:
{}
[llm/start] [1:chain:batch > 2:llm:WatsonxLLM] Entering LLM run with input:
{
  "prompts": [
    "Human: \n\nRewrite the input into valid json\n\n\nInput:\n{\n    \"name\": \"John Doe\",\n    \"age\": 30,\n    \"isStudent\": false\n    \"address\": {\n        \"street\": \"123 Main St\",\n        \"city\": \"Anytown\",\n        \"state\": \"CA\",\n    }\n    \"hobbies\": [\"reading\", \"swimming\", \"cycling\"]\n}\nOutput:\n{\n    \"name\": \"John Doe\",\n    \"age\": 30,\n    \"isStudent\": false,\n    \"address\": {\n        \"s

...

[chain/start] [1:chain:batch] Entering Chain run with input:
{}
[llm/start] [1:chain:batch > 2:llm:WatsonxLLM] Entering LLM run with input:
{
  "prompts": [
    "Human: Please extract relevant sentences from the provided context that is absolutely required answer the following question. If no relevant sentences are found, or if you believe the question cannot be answered from the given context, return the phrase \"Insufficient Information\".  While extracting candidate sentences you're not allowed to make any changes to sentences from given context.\n\nquestion:How to deposit a cheque issued to an associate in my business into my business account?\ncontext:\n[\n[\n[\ncandidate sentences:"
  ]
}
[llm/start] [1:chain:batch > 3:llm:WatsonxLLM] Entering LLM run with input:
{
  "prompts": [
    "Human: Please extract relevant sentences from the provided context that is absolutely required answer the following question. If no relevant sentences are found, or if you believe the question canno

100%|██████████| 1/1 [00:10<00:00, 10.50s/it]

[llm/end] [1:chain:batch > 2:llm:WatsonxLLM] [10.47s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\"You can deposit a cheque issued to an associate in your business into your business account by",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[llm/end] [1:chain:batch > 3:llm:WatsonxLLM] [10.47s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "[\n\"USPS offers money orders for purchase at any post office location.\"\n\"Money",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[llm/end] [1:chain:batch > 4:llm:WatsonxLLM] [10.47s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. EIN stands for Employer Identification Number.\n2. An EIN is",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "r

{'faithfulness': nan, 'context_precision': nan, 'context_relevancy': 0.5000}
Scoring completed!


/opt/conda/envs/Python-RT23.1/lib/python3.10/site-packages/ragas/evaluation.py:130: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])


# Conclusion
What makes HyDE intriguing is its ability to perform effectively without the need for relevant labels. It offloads the task of modelling relevance from traditional retrieval models to a language model that can generalize to a wide range of queries and tasks. This approach has several advantages:

Zero-Shot Retrieval: HyDE can work “out of the box” without relying on a large dataset of labeled examples.
Cross-Lingual: It performs well across various languages, making it suitable for multilingual search applications.
Flexibility: HyDE’s approach allows it to adapt to different tasks without extensive fine-tuning.
HyDE marks a significant advancement in retrieval techniques, providing a novel approach by focusing on answer-to-answer embedding similarity.

# Publish the model to deployment space, and deploy the model from deployment space to cloud

In [68]:
# Find and set the deployment space_id

from ibm_watson_machine_learning import APIClient
api_key = '14qK2UldM1i25REfFm3HcH9y23_NVL-6taz3pgKjrnC9'
location = 'us-south'
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}
client = APIClient(wml_credentials)
print(client)
client.spaces.list(limit=10)

------------------------------------  ----------------------------------  ------------------------
ID                                    NAME                                CREATED
936fb5a6-9870-4b60-840b-fd9dd49b9b9c  RAG_with_HYDE_approach dep-space    2024-01-14T17:02:06.030Z
19c1e548-0900-4c48-ae6f-fbe69d3027ab  AI governance dep-space             2024-01-13T16:17:24.928Z
20ed5b58-d81f-4051-99d8-b746d25d3eeb  Deploy from Notebook                2024-01-13T07:26:54.667Z
2c0861cf-c1a7-4a2d-89e3-725cb8d66bad  Build an AI model dep-space         2024-01-12T18:43:30.546Z
b9f03ed8-b89f-4308-a1dc-beb0d2880bb5  AI governance deployment space      2024-01-09T04:00:30.115Z
402a182b-9f1f-44f4-8703-aa4d7fd5ec2c  Build an AI model deployment space  2024-01-09T02:17:04.615Z
b737aae3-3481-447f-8cf0-d0933ce61b7a  deploy notebook deployment space    2024-01-08T23:55:33.759Z
6a305bf6-0f77-42de-bc1d-03b8d2d0c82f  kidney_diseases_deployment_space    2023-12-26T16:29:11.962Z
93027de1-01bc-4522-9821-a6d

,ID,NAME,CREATED
0,936fb5a6-9870-4b60-840b-fd9dd49b9b9c,RAG_with_HYDE_approach dep-space,2024-01-14T17:02:06.030Z
1,19c1e548-0900-4c48-ae6f-fbe69d3027ab,AI governance dep-space,2024-01-13T16:17:24.928Z
2,20ed5b58-d81f-4051-99d8-b746d25d3eeb,Deploy from Notebook,2024-01-13T07:26:54.667Z
3,2c0861cf-c1a7-4a2d-89e3-725cb8d66bad,Build an AI model dep-space,2024-01-12T18:43:30.546Z
4,b9f03ed8-b89f-4308-a1dc-beb0d2880bb5,AI governance deployment space,2024-01-09T04:00:30.115Z
5,402a182b-9f1f-44f4-8703-aa4d7fd5ec2c,Build an AI model deployment space,2024-01-09T02:17:04.615Z
6,b737aae3-3481-447f-8cf0-d0933ce61b7a,deploy notebook deployment space,2024-01-08T23:55:33.759Z
7,6a305bf6-0f77-42de-bc1d-03b8d2d0c82f,kidney_diseases_deployment_space,2023-12-26T16:29:11.962Z
8,93027de1-01bc-4522-9821-a6d89a4a4968,ElectricityHourly_deployment_space,2023-12-22T16:23:59.677Z
9,4f9a28e0-17d1-42cf-86ae-01ab677cde06,Car_Evaluation_deployment_space,2023-12-22T15:50:52.127Z


In [72]:
space_id = '936fb5a6-9870-4b60-840b-fd9dd49b9b9c'
client.set.default_space(space_id)

'SUCCESS'

In [78]:
# Publish the model to deployment space with space_id = '936fb5a6-9870-4b60-840b-fd9dd49b9b9c'
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-23.1-py3.10")
print('sofware_spec_uid: ', sofware_spec_uid) # 336b29df-e0e1-5e7d-b6a5-f6ab722625b2
metadata = {
            client.repository.ModelMetaNames.NAME: 'RAG_with_HYDE_approach model',
            client.repository.ModelMetaNames.TYPE: 'RAG_with_HYDE_approach model_1.1',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

# published_model = client.repository.store_model(
#     model=model,
#     meta_props=metadata,
#     training_data=train_data,
#     training_target=train_labels)

# # Get model details
# import json
# published_model_uid = client.repository.get_model_id(published_model)
# model_details = client.repository.get_details(published_model_uid)
# print(json.dumps(model_details, indent=2))

sofware_spec_uid:  336b29df-e0e1-5e7d-b6a5-f6ab722625b2


In [79]:
# Deploy the model in a Cloud

models_details = client.repository.list_models()

if len(models_details) > 0:
    metadata = {
        client.deployments.ConfigurationMetaNames.NAME: "Deployment of RAG_with_HYDE_approach model",
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }

    created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



--  ----  -------  ----  ----------  ----------------
ID  NAME  CREATED  TYPE  SPEC_STATE  SPEC_REPLACEMENT
--  ----  -------  ----  ----------  ----------------
